<img src="https://github.com/seap-udea/xearch/blob/main/resources/xearch-logo.png?raw=true" align="left">

<a target="_blank" href="https://colab.research.google.com/github/seap-udea/xearch/blob/main/public/xearch-rv-findmass.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Radial velocity method
## Orbit, mass and residuals

In this notebook we will play with radial velocity method. In particular, we will study the dependency of the signal on planetary mass, inclination and eccentricity.

**Instructions:**
- Start by running all cells. It may take a while since the script must install all required packages and download exoplanet data.
- Select a planetary system.
- Once selected the planetary system, run again the cell corresponding to the analysis.

In [1]:
#@title Select a planetary system
## This cell is hidden in Google Colab
# Install dependencies
import os,sys
if 'google.colab' in sys.modules:
    # Google colab only needs unusual packages
    !pip install -q unzip astropy spiceypy
else:
    # For running in windows or linux environments all this is required
    !pip install -q unzip gdown numpy scipy matplotlib astropy pandas ipywidgets spiceypy

# Downloading signal
if not os.path.isdir('signals'):
    # Get data
    print("Getting signals for the first time")
    
    # Available at: https://drive.google.com/file/d/1UucCokiBR6sR8HlVVt6ewBlYX9YoUwbf/view?usp=drive_link
    if 'google.colab' in sys.modules:
      !gdown -q 1UucCokiBR6sR8HlVVt6ewBlYX9YoUwbf
    else:
      !python3 -m gdown -q 1UucCokiBR6sR8HlVVt6ewBlYX9YoUwbf

    #!python -m unzip -oq signals.zip
    import zipfile
    with zipfile.ZipFile('signals.zip', 'r') as zip_ref:
        zip_ref.extractall()

# External modules
import spiceypy as spy
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as c
import pandas as pd
from ipywidgets import interact, widgets, interactive, fixed, interact_manual, Layout
import glob
import pickle
from scipy.interpolate import interp1d

# Constants
G = c.G.value
Msun = c.M_sun.value
Mearth = c.M_earth.value
Mjup = c.M_jup.value
Rsun = c.R_sun.value
Rearth = c.R_earth.value
Rjup = c.R_jup.value
Mjup2earth = Mjup/Mearth
Rjup2earth = Rjup/Rearth

# Units
K = 1 # Kelvin
days = 86400
JD = days # Julian days
BJD = days # Bessel Julian days
years = 365.25*days # Julian years
au = c.au.value
deg = np.pi/180
rad = 1/deg
Gyr = 1e9*years
dex = 1

# Routines
def unpickle(filepath):
    with open(filepath, 'rb') as file:
        data = pickle.load(file)
    return data

class System(object):
    pass

global_selection = None

# Read list of planetary systems
planetary_systems = glob.glob('signals/rv/*.pkl')
planetary_systems = sorted([x.split('/')[-1].split('.')[0].replace('_',' ') for x in glob.glob('signals/rv/*.pkl')])

# Read selection of special planetary system

# Single
filepath = 'signals/rv/selections/selection-rv-single.txt'
with open(filepath, 'r') as file:
    lines = file.readlines()
single_systems = [line.strip().split('.')[0].replace('_',' ') for line in lines]

# Multiple
filepath = 'signals/rv/selections/selection-rv-multiple.txt'
with open(filepath, 'r') as file:
    lines = file.readlines()
multiple_systems = [line.strip().split('.')[0].replace('_',' ') for line in lines]

# Eccentric
filepath = 'signals/rv/selections/selection-rv-eccentric.txt'
with open(filepath, 'r') as file:
    lines = file.readlines()
eccentric_systems = [line.strip().split('.')[0].replace('_',' ') for line in lines]

# Anomalous
filepath = 'signals/rv/selections/selection-rv-anomalous.txt'
with open(filepath, 'r') as file:
    lines = file.readlines()
anomalous_systems = [line.strip().split('.')[0].replace('_',' ') for line in lines]

# Compile
planetary_systems = \
    ['-- Single systems --'] + single_systems + \
    ['-- Multiple systems --'] + multiple_systems + \
    ['-- Eccentric systems --'] + eccentric_systems + \
    ['-- Anomalous systems --'] + anomalous_systems + \
    ['-- All systems --'] + planetary_systems

# Select planetary system
cadence = {
    'frequent' : 100,
    'medium' : 50,
    'sparse' : 10,
    'very sparse': 5,
}
error = dict(
    small = 0.05,
    medium = 0.1,
    large = 0.2,
)

# Select planetary system
system = None
system_old = None
def select_planetary_system(
      planetary_system,
      sampling_cadence,
      sampling_error,
      number_of_periods,
    ):
    global system
    global system_old
    global global_selection

    # Filepath
    system_filepath = f"signals/rv/{planetary_system.replace(' ','_')}.pkl"
    if os.path.isfile(system_filepath):
        system = unpickle(f"signals/rv/{planetary_system.replace(' ','_')}.pkl")
    else:
        print(f"No system selected yet")
        return

    # Interpolated radial velocity
    system.tfactor = system.UT/days
    vrad_fun = interp1d(system.tobs,system.vrads,kind='cubic')
    system.tmin = 0.0
    system.tmax = system.Pmax*(number_of_periods)
    system.duration = system.tmax - system.tmin
    # Create time and velocity array
    ts_nominal = np.linspace(system.tmin,system.tmax,number_of_periods*cadence[sampling_cadence])
    vs_nominal = vrad_fun(ts_nominal)
    # Create the signal
    system.ts_signal = ts_nominal
    # system.dvs_signal = abs(error[sampling_error]*vs_nominal)
    system.dvs_signal = system.Kmax*error[sampling_error]
    system.vrads_signal = np.random.normal(vs_nominal,system.dvs_signal)

    if planetary_system != system_old:
        # Reset global selection
        global_selection = None

    system_old = planetary_system
    # Message
    print(f"Signal with {sampling_cadence} cadence and {sampling_error} errors for {number_of_periods} periods, generated for system '{planetary_system}'.")

planetary_system = interact(
    select_planetary_system,
    planetary_system=widgets.Dropdown(
        options=planetary_systems,
        value=single_systems[0],
        description='System:',
        disabled=False,
    ),
    sampling_cadence=widgets.Dropdown(
        options=['frequent', 'medium', 'sparse','very sparse'],
        description='Cadence:',
        disabled=False,
    ),
    sampling_error=widgets.Dropdown(
        options=['small', 'medium', 'large'],
        description='Errors:',
        disabled=False,
    ),
    number_of_periods=widgets.Dropdown(
        options=[2,3,4,5],
        value = 4,
        description='Duration:',
        disabled=False,
    ),
)

interactive(children=(Dropdown(description='System:', index=1, options=('-- Single systems --', '51 Peg', '3 C…

In [11]:
#@title Find the orbital period of the planet
## This cell is hidden in Google Colab
def plot_signal(
        tini, # in days
        gperiod, # in days
        gmass, # in Mearth
        ge, # Eccentricity
        ginc, # in degrees
        gMini, # in degrees
        qsignal, # show orbit?
        qfolding, # show folding?
    ):
    # Global selection
    global global_selection
    
    global_selection = dict(
        tini = tini,
        gperiod = gperiod,
        gmass = gmass,
        ge = ge,
        ginc = ginc,
        gMini = gMini,
        qsignal = qsignal,
        qfolding = qfolding,
    )

    # Convert to internal units
    tini /= system.tfactor
    gperiod /= system.tfactor
    system.G = G / (system.UL**3/(system.UM*system.UT**2))

    # Plotting options
    errorbar_options = dict(color='k',fmt='o',ms=3,capsize=2)

    # Select number of panels
    nplots = 1
    if qfolding:
        nplots += 1
    nplots = 3

    # Plot Data
    fig,axs = plt.subplots(nplots,1,figsize=(10,2*nplots),gridspec_kw=dict(height_ratios=[3,1,3]))
    axs = [axs] if not isinstance(axs,np.ndarray) else axs

    # Plot signal
    ax = axs[0]
    ax.errorbar(system.ts_signal*system.UT/days,
                system.vrads_signal*system.UV,
                system.dvs_signal*system.UV,
                **errorbar_options)
    ax.set_xlabel(f"Time [days]")
    ax.set_ylabel(f"Radial Velocity [m/s]")

    # Plot periodic lines
    tmarks = np.arange(tini,system.tmax,gperiod)
    for tm in tmarks:
        ax.axvline(tm*system.tfactor,ls='--',color='r')

    # Decoration
    ax.grid()
    ax.set_xlim((system.tmin-system.duration/100)*system.tfactor,
                (system.tmax+system.duration/100)*system.tfactor)

    # Title
    ax.set_title(rf"Finding period of {system.name} ($M_\star = {system.mstar:.2f}\;M_\odot$): $P$ (guess) = {gperiod*system.tfactor:.2f} days",fontsize=13)

    # Plot theoretical signal
    if qsignal:
        # Orbital parameters
        mu = G*system.mstar*system.UM
        n = 2*np.pi/(gperiod*system.tfactor*days)
        a = (mu/n**2)**(1/3)/system.UL
        # Other orbital elements
        e = ge
        q = a*(1-e)
        inc = ginc*deg
        omega = system.planets[0]['omega']
        Omega = system.planets[0]['Omega'] 
        M = gMini*deg # system.planets[0]['M']+ np.pi
        m = gmass*Mearth/system.UM

        # Generate the theoretical signal
        Nt = 100
        ts = np.linspace(system.tmin,system.tmax,Nt)
        Xs = np.zeros((Nt,6))
        mu = system.G*system.mstar
        elements = [q, e, inc, Omega, omega, M, 0, mu]
        for i,t in enumerate(ts):
            Xs[i] = m/system.mstar*spy.conics(elements, t)
        vrad_teos = -Xs[:,5]
        ax.plot(ts*system.UT/days,vrad_teos*system.UV,'b',lw=3,alpha=0.3,zorder=-100)
        vmin,vmax = vrad_teos.min(),vrad_teos.max() 
        ax.axhline(vmin*system.UV,color='b',alpha=0.3)
        ax.axhline(vmax*system.UV,color='b',alpha=0.3)
        
        if qfolding:
            # Fold signal
            ax = axs[2]
            phases_teo = ((ts - tini) % gperiod)/gperiod
            ind = np.argsort(phases_teo)
            ax.plot(phases_teo[ind],vrad_teos[ind]*system.UV,'b-',lw=3,alpha=0.3,zorder=-100)
            ax.set_xlim(-0.01,1.01)

    # Plot residuals
    ax = axs[1]
    ax.sharex(axs[0])
    ax.set_xlabel(f"Time [days]",color='w')
    ax.set_ylabel(f"Residual",color='w')        
    if qsignal:
        vrad_interp = interp1d(ts,vrad_teos,kind='cubic')
        res = vrad_interp(system.ts_signal) - system.vrads_signal
        ax.errorbar(system.ts_signal*system.UT/days,res*system.UV,system.dvs_signal*system.UV,**errorbar_options)
        
        ax.xaxis.label.set_color('k')
        ax.yaxis.label.set_color('k')
    else:
        ax.tick_params(axis='x', colors='w')
        ax.tick_params(axis='y', colors='w')
        for spine in ax.spines.values():
            spine.set_visible(False)

    # Plot folded
    ax = axs[2]
    ax.set_xlabel(f"Phase",color='w')
    ax.set_ylabel(f"Radial Velocity [m/s]",color='w')
    if qfolding:
        # Fold signal
        phases = ((system.ts_signal - tini) % gperiod)/gperiod
        # Plot folded signal
        ax.errorbar(phases,
                    system.vrads_signal*system.UV,
                    system.dvs_signal*system.UV,
                    **errorbar_options)
        ax.set_xlim(-0.01,1.01)
        ax.xaxis.label.set_color('k')
        ax.yaxis.label.set_color('k')
        ax.grid()
    else:
        ax.tick_params(axis='x', colors='w')
        ax.tick_params(axis='y', colors='w')
        for spine in ax.spines.values():
            spine.set_visible(False)

    fig.tight_layout()
    plt.show()

if global_selection is None:
    global_selection = dict(
        tini = round(system.tmin*system.tfactor,2),
        gperiod = round(system.duration*system.tfactor/10,2),
        gmass = round(1e-3*system.mstar*system.UM/Mearth,2),
        ge = 0,
        ginc = round(90,2),
        gMini = round(system.planets[0]['M']*rad,2),
        qsignal = False,
        qfolding = False,
    )

step_time = round(system.duration*system.tfactor/200,1)
avg_mass = np.round(np.mean([pl['m'] for pl in system.planets])*system.UM/Mearth,2)
step_mass = avg_mass/100
interact(
    plot_signal,
    tini=widgets.BoundedFloatText(
        description='Initial time:',
        min=system.tmin*system.tfactor,max=system.tmax*system.tfactor,step=step_time,value=global_selection['tini'],
        redout_format = '.4f',
        continuous_update=False
    ),
    gperiod=widgets.BoundedFloatText(
        description='Period: ',
        min=0,max=system.nperiods*system.Pmax*system.tfactor,step=step_time,value=global_selection['gperiod'],
        redout_format = '.1f',
        continuous_update=False
    ),
    gmass=widgets.BoundedFloatText(
        description='Planet mass: ',
        min=0,max=5*avg_mass,step=step_mass,value=global_selection['gmass'],
        redout_format = '.1f',
        continuous_update=False
    ),
    ge=widgets.BoundedFloatText(
        description='Eccentricity: ',
        min=0,max=1,step=0.01,value=global_selection['ge'],
        redout_format = '.3f',
        continuous_update=False
    ),
    ginc=widgets.BoundedFloatText(
        description='Inclination: ',
        min=0,max=90,step=0.01,value=global_selection['ginc'],
        redout_format = '.1f',
        continuous_update=False
    ),
    gMini=widgets.BoundedFloatText(
        description='Position: ',
        min=0,max=360,step=0.01,value=global_selection['gMini'],
        redout_format = '.1f',
        continuous_update=False
    ),
    qfolding=widgets.Checkbox(
        description='Fold the signal',value=global_selection['qfolding'],
        continuous_update=False
    ),
    qsignal=widgets.Checkbox(
        description='Show the signal',value=global_selection['qsignal'],
        continuous_update=False
    ),
    layout=Layout(display='flex', flex_flow='row', justify_content='space-between', align_items='center'),
);

interactive(children=(BoundedFloatText(value=0.0, description='Initial time:', max=3341.9079999999994, step=16…

-----
*Designed and maintained by [Jorge I. Zuluaga](mailto:jorge.zuluaga@udea.edu.co) (2024)*